# Introduction

This workout comes from [2023 Week 16 | Power BI: Measure Format Strings and Conditionally Formatted Titles](https://workout-wednesday.com/pbi-2023-w16/), the main focus of which is a dashboard allowing the user to choose the number of top products to be included in the report and the metric to be used in the report visuals. 

In [3]:
import pandas as pd

In [4]:
file_path = '../data/Dataset-Customer Profitability.xlsx'
data = pd.read_excel(file_path, sheet_name=None)
data.keys()

dict_keys(['state', 'executive', 'industry', 'customer', 'product', 'scenario', 'date', 'bu', 'factsales'])

In [5]:
def get_data(file_path):
    data = pd.read_excel(file_path, sheet_name=None)
    customers = data['customer'].merge(
        data['industry'], left_on='Industry ID', right_on='ID', how='left').merge(
        data['state'], left_on='State', right_on='StateCode', how='left')
    data_customer = data['factsales'].sort_values(by='YearPeriod').reset_index(drop=True).merge(
        data['product'], on='Product Key', how='left').merge(
        customers, left_on='Customer Key', right_on='Customer', how='left').merge(
        data['date'], on='YearPeriod', how='left')
    data_customer['COGS'] = data_customer.iloc[:,range(6, 12)].sum(axis=1)
    data_customer['Gross Margin'] = data_customer['Revenue'] - data_customer['COGS']
    data_selected = data_customer.loc[:, ['YearPeriod', 'Customer Key', 'Industry', 'Product', 'State_y', 'COGS', 'Gross Margin', 'Year', 'Qtr', 'Month']]
    data_selected['Product'] = data_selected['Product'].fillna('(Blank)')
    data_selected['Industry'] = data_selected['Industry'].fillna('(Blank)')
    data_selected['Month'] = pd.Categorical(data_selected['Month'].values, 
                                        categories=['Jan', 'Feb', 'Mar', 'Apr', 
                                                    'May', 'Jun', 'Jul', 'Aug',
                                                    'Sep', 'Oct', 'Nov', 'Dec'],
                                        ordered=True)
    return data_selected.astype({'Year': 'str'})

In [6]:
top_num = 3
metrics = '# of Customers'

In [7]:
import bokeh.plotting as bp
import bokeh.io as bio
import bokeh.models as bm
import bokeh.layouts as bl

bio.output_notebook()

Loading BokehJS ...

In [91]:
# def plot_data(top_num, metrics):
data_selected = get_data(file_path)
if metrics == '# of Customers':
    field = 'Customer Key'
    table_data = data_selected.groupby('Product', dropna=False).nunique().sort_values(by=field, ascending=False)[[field]]
    indx = table_data.index[:top_num]
    filtered_rows = data_selected['Product'].apply(lambda x: x in indx)
    bar_data = data_selected.loc[filtered_rows, :].groupby('Industry', dropna=False).nunique().sort_values(by=field)[[field]]
    line_group = data_selected.loc[filtered_rows, :].groupby(['Year', 'Qtr', 'Month'], dropna=False, observed=True)
    line_data = line_group.nunique()[[field]]
    state_data = data_selected.loc[filtered_rows, :].groupby('State_y').nunique()[[field]]
    
source = bm.ColumnDataSource(data_selected)
source_table = bm.ColumnDataSource(table_data)
source_bar = bm.ColumnDataSource(bar_data)
source_line = bm.ColumnDataSource(line_data)
source_map = bm.ColumnDataSource(state_data)

# Table chart
columns = [
    bm.TableColumn(field='Product', title='Product'),
    bm.TableColumn(field=field, title='Value')]
chart_table = bm.DataTable(
    columns=columns, source=source_table, 
    width=250, height=150,
    index_position=None,
    view=bm.CDSView(filter=bm.IndexFilter(list(range(top_num)))))

# Line chart
chart_line = bp.figure(width=700, height=250, 
                       x_range=bm.FactorRange(*source_line.data['Year_Qtr_Month'], group_padding=0, subgroup_padding=0),
                      tooltips=[("Month", "@Year_Qtr_Month"), ("Value", f"@{{{field}}}")])
chart_line.line(x='Year_Qtr_Month', y=field, source=source_line)

# Bar chart
char_bar = bp.figure(width=350, height=500, y_range=source_bar.data['Industry'],
                    tooltips=[("Industry", "@Industry"), ("Value", f"@{{{field}}}")])
char_bar.hbar(y='Industry', right=field, source=source_bar,
              line_color='white')

# Map chart


bio.show(bl.gridplot([[chart_table, chart_line], [char_bar, None]]))

In [102]:
p = figure(x_range=(-14000000, -7000000), y_range=(2700000, 6400000),
           width=600, height=300,
           x_axis_type="mercator", y_axis_type="mercator"
          )

p.add_tile("CartoDB Positron", retina=True)

show(p)

# for reference https://geopandas.org/en/stable/gallery/plotting_basemap_background.html#Matching-coordinate-systems

In [101]:
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'